In [2]:
pip install PyMuPDF

     |████████████████████████████████| 6.4 MB 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
jj=''

In [32]:
fileName = '1708.08021.pdf'
textList = [
    '''14 are guaranteed to have those properties.''',
    '''his paper we present the design and implementation of''',
    '''Higher-order functions (lines 1–4) are quite common in JavaScript.''',
    '''invalidated by assignments, which can even happen indirectly via calls (line 26).'''
]

In [42]:
fileName = '1604.02480.pdf'
textList = [
  '''Second, RSC accounts for dynamic typing by using a recently proposed technique called''',
  '''specify the function’s precon''',
  '''synthesize the instantiations of (2). In a nutshell,''',
  '''synthesize the instantiations of (2). In a nutshell,'''
]

In [43]:
import fitz

### READ IN PDF
doc = fitz.open(fileName)
cordinates = []

for text in textList:
    matchCount=0
    matchData = ''
    pageCount = 0
    for page in doc:
        pageCount = pageCount+1
        
        ### SEARCH
        text_instances = page.searchFor(text)
        
        if(len(text_instances)>0):
            print(text_instances)
            print('\n')
            matchCount = matchCount+1
            print(pageCount)
            matchData = text_instances
        ### HIGHLIGHT
        for inst in text_instances:
            highlight = page.addHighlightAnnot(inst)
            highlight.update()
            
        if(len(text_instances)>0):
            break
    
    #Check if more than 1 match
    if(matchCount>1):
        print('---MULTIPLE MATCHES----')
    elif(matchCount==1):
        cordinates.append({
            "question": text,
            "rectList" : matchData,
            "pageNumber": pageCount
        })

### OUTPUT
doc.save("temp.pdf", garbage=4, deflate=True, clean=True)

[Rect(414.11993408203125, 240.66900634765625, 519.6779174804688, 253.95916748046875), Rect(519.6779174804688, 241.68519592285156, 556.3032836914062, 253.84957885742188), Rect(317.039794921875, 253.68519592285156, 342.0611267089844, 265.8495788574219), Rect(342.0611267089844, 252.66900634765625, 534.4208984375, 265.95916748046875)]


1
[Rect(173.7606964111328, 366.18963623046875, 289.98193359375, 379.47979736328125)]


2
[Rect(342.1201477050781, 376.2055358886719, 382.54351806640625, 388.3699035644531), Rect(382.54351806640625, 375.1893310546875, 538.290771484375, 388.4794921875)]


3
[Rect(342.1201477050781, 376.2055358886719, 382.54351806640625, 388.3699035644531), Rect(382.54351806640625, 375.1893310546875, 538.290771484375, 388.4794921875)]


3


In [44]:
width = doc[0].MediaBox[2]
height =doc[0].MediaBox[3]

In [45]:
cordinates

[{'question': 'Second, RSC accounts for dynamic typing by using a recently proposed technique called',
  'rectList': [Rect(414.11993408203125, 240.66900634765625, 519.6779174804688, 253.95916748046875),
   Rect(519.6779174804688, 241.68519592285156, 556.3032836914062, 253.84957885742188),
   Rect(317.039794921875, 253.68519592285156, 342.0611267089844, 265.8495788574219),
   Rect(342.0611267089844, 252.66900634765625, 534.4208984375, 265.95916748046875)],
  'pageNumber': 1},
 {'question': 'specify the function’s precon',
  'rectList': [Rect(173.7606964111328, 366.18963623046875, 289.98193359375, 379.47979736328125)],
  'pageNumber': 2},
 {'question': 'synthesize the instantiations of (2). In a nutshell,',
  'rectList': [Rect(342.1201477050781, 376.2055358886719, 382.54351806640625, 388.3699035644531),
   Rect(382.54351806640625, 375.1893310546875, 538.290771484375, 388.4794921875)],
  'pageNumber': 3},
 {'question': 'synthesize the instantiations of (2). In a nutshell,',
  'rectList': 

In [46]:
import json

annoCount=0
for anno in cordinates:
    annoCount = annoCount+1
    question = anno['question']
    finalDictionary = {
        
        "content": {
            "text": question
        },
        
        "position" :{
            "boundingRect":{},
            "rects":[],
            
            'pageNumber': anno['pageNumber'],
            
            
        },
        "comment": {
            "text": 'Quest',
            "emoji": ""
          },
          "id": str(annoCount)
    }
    jsonString = ''
    
    smallestX1 = 999999
    biggestX2 = -1
    
    smallestY1 = 999999
    biggestY2 = -1
    
    for rect in anno['rectList']:
        dictionary = {'x1':float(rect[0]), 'y1':float(rect[1]), 'x2':float(rect[2]), 'y2':float(rect[3]), 'width':float(width), 'height':float(height)}
        finalDictionary["position"]['rects'].append(dictionary)
        
        smallestX1 = rect[0] if rect[0]<smallestX1 else smallestX1
        biggestX2 = rect[1] if rect[1]>biggestX2 else biggestX2
        
        smallestY1 = rect[2] if rect[2]<smallestY1 else smallestY1
        biggestY2 = rect[3] if rect[3]>biggestY2 else biggestY2
    
    finalDictionary["position"]['boundingRect']['x1'] = smallestX1
    finalDictionary["position"]['boundingRect']['x2'] = biggestX2
    finalDictionary["position"]['boundingRect']['y1'] = smallestY1
    finalDictionary["position"]['boundingRect']['y2'] = biggestY2
    
    finalDictionary["position"]['boundingRect']['width'] = float(width)
    finalDictionary["position"]['boundingRect']['height'] = float(height)

    jsonString = json.dumps(finalDictionary, indent=4)
    print(jsonString+',')

{
    "content": {
        "text": "Second, RSC accounts for dynamic typing by using a recently proposed technique called"
    },
    "position": {
        "boundingRect": {
            "x1": 317.039794921875,
            "x2": 253.68519592285156,
            "y1": 342.0611267089844,
            "y2": 265.95916748046875,
            "width": 612.0,
            "height": 792.0
        },
        "rects": [
            {
                "x1": 414.11993408203125,
                "y1": 240.66900634765625,
                "x2": 519.6779174804688,
                "y2": 253.95916748046875,
                "width": 612.0,
                "height": 792.0
            },
            {
                "x1": 519.6779174804688,
                "y1": 241.68519592285156,
                "x2": 556.3032836914062,
                "y2": 253.84957885742188,
                "width": 612.0,
                "height": 792.0
            },
            {
                "x1": 317.039794921875,
                "y

In [11]:
def _extract_annot(annot, words_on_page):
    """Extract words in a given highlight.

    Args:
        annot (fitz.Annot): [description]
        words_on_page (list): [description]

    Returns:
        str: words in the entire highlight.
    """
    quad_points = annot.vertices
    quad_count = int(len(quad_points) / 4)
    sentences = ['' for i in range(quad_count)]
    for i in range(quad_count):
        points = quad_points[i * 4: i * 4 + 4]
        words = [
            w for w in words_on_page if
            _check_contain(fitz.Rect(w[:4]), points)
        ]
        sentences[i] = ' '.join(w[4] for w in words)
    sentence = ' '.join(sentences)

    return sentence

_threshold_intersection = 0.7  # if the intersection is large enough.


def _check_contain(r_word, points):
    """If `r_word` is contained in the rectangular area.

    The area of the intersection should be large enough compared to the
    area of the given word.

    Args:
        r_word (fitz.Rect): rectangular area of a single word.
        points (list): list of points in the rectangular area of the
            given part of a highlight.

    Returns:
        bool: whether `r_word` is contained in the rectangular area.
    """
    # `r` is mutable, so everytime a new `r` should be initiated.
    r = fitz.Quad(points).rect
    r.intersect(r_word)

    if r.getArea() >= r_word.getArea() * _threshold_intersection:
        contain = True
    else:
        contain = False
    return contain


In [12]:
doc = fitz.open('./temp.pdf')


In [13]:
for page in doc.pages():
    print(page)
    #page = doc[0]
    annot = page.annots()       #page.firstAnnot
    annot2 = page.annots()  
    words = page.getText("words")
    annotIterVal = sum(1 for x in annot2)
    pageOutlist = []
    for i in range(annotIterVal):
        thisAnnot = annot.__next__()
        #print(thisAnnot)
        finalout = _extract_annot(thisAnnot, words)
        #print(finalout)
        pageOutlist.append(finalout)
        #print('\n')
        

    print(pageOutlist)
    print('\n')

page 0 of ./temp.pdf
['this paper we present the design and implementation of']


page 1 of ./temp.pdf
['Higher-order functions (lines 1–4) are quite common in JavaScript.']


page 2 of ./temp.pdf
['14 are guaranteed to have those properties.', 'invalidated', 'by assignments, which can even happen', 'indirectly via calls (line 26).']


page 3 of ./temp.pdf
[]


page 4 of ./temp.pdf
[]


page 5 of ./temp.pdf
[]


page 6 of ./temp.pdf
[]


page 7 of ./temp.pdf
[]


page 8 of ./temp.pdf
[]


page 9 of ./temp.pdf
[]


page 10 of ./temp.pdf
[]


page 11 of ./temp.pdf
[]


page 12 of ./temp.pdf
[]


page 13 of ./temp.pdf
[]


page 14 of ./temp.pdf
[]


page 15 of ./temp.pdf
[]


page 16 of ./temp.pdf
[]


page 17 of ./temp.pdf
[]


page 18 of ./temp.pdf
[]


page 19 of ./temp.pdf
[]


page 20 of ./temp.pdf
[]


page 21 of ./temp.pdf
[]


page 22 of ./temp.pdf
[]


page 23 of ./temp.pdf
[]


page 24 of ./temp.pdf
[]


page 25 of ./temp.pdf
[]


page 26 of ./temp.pdf
[]


page 27 of ./temp.pdf